### Specs, Ctrls and values are STR type in Epogen's dataset.
### NEED to convert to numeric on the epogen_quan dataset

### INPUT ISSUE: illegal specs(containing <= in UAL), values/LAL are normal
currently using RE(regular expression) to address this, not sure if re solves all problems

In [1]:
import re

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import anderson
from scipy.stats import kstest
from scipy.stats import shapiro

convert the map/filter/reduce into a list, for readability.
list(map(....)) otherwise they are just map/filter objects which can not be displayed.


##### STR to NUM on a series
###### 1. str.isdigit(): a function only applied to string, and judge based on the string altogether. Does not judge if there is any digit contained in the string.
eg: "100"-->T, "100+"-->F

A way to exam digit element-wise: use filter(treat a string not as a whole but as a list of elements)

list(filter(str.isdigit, '200 grams'))  output:["2","0","0"]

concat all elements in a list: ''.join(list(filter(str.isdigit, '200 grams')))

convert this to a number: float(''.join(list(filter(str.isdigit, '200 grams'))))

This function can be mapped to UAL/LAL for str->num conversion ONLY IF: 
    There is only one number in each cell. There has to be at least one digit for "float" function to work.

###### 2. re.findall
return all blocks of number

re.findall('\d+', "00>=6000asdf90.sdf9")-->['00', '6000', '90', '9']

return all sigle digits in a string(as a list)

re.findall('\d', "00>=6000asdf90.sdf9")-->['0', '0', '6', '0', '0', '0', '9', '0', '9']

Requires one number in each cell.

In [44]:
def string_to_num(input_string):
    output=''.join(list(filter(str.isdigit, input_string)))
    if (output==""): 
        return(".")
    else:
        return(float(output))

In [126]:
re.findall('\d+',"-0n3")[0]

'0'

In [ ]:
def string_to_num_re(intput_string):
    output_list=re.findall('\d+',input_string)
    if (output_list==[]):
        #if the original cell/spec is missing/unspecified
        return(".")
    else:
        #take the only number in the list
        return(float(output_list[0]))

In [55]:
def specs_num_conversion(df):
    
    df["UAL"]=pd.Series(map(string_to_num,df["UAL"]))
    df["LAL"]=pd.Series(map(string_to_num,df["LAL"]))

In [69]:
file_path="query-Aranesp-APPX-OCT2.csv"
df=pd.read_csv(file_path,index_col=False)

In [70]:
df=df.rename(columns={"ual":"UAL","lal":"LAL","lcl":"LCL","ucl":"UCL"})

In [60]:
specs_num_conversion(df)

In [120]:
''.join(list(filter(str.isdigit, int(10))))

TypeError: 'int' object is not iterable

pd.to_numeric("adsf","coerce")-->NAN
pd.to_numeric("?", errors='ignore')-->"?"  the string itself(no change on strings)

In [26]:
pd.to_numeric("?", errors='ignore')

'?'

In [3]:
epogen=pd.read_csv("APPX-Epogen-XMLRawData.csv")

In [ ]:
pd.to_numeric(s, errors='ignore')
pd.to_numeric(s, errors='coerce')

In [37]:
''.join(list(filter(str.isdigit, '.')))

''

In [4]:
#delete the empty rows at the end of the file
epogen=epogen.dropna(how="all")

In [23]:
epogen.columns
#all capitalized column names, no need to change

Index(['UNIT PROCEDURE', 'SAMPLING POINT', 'PARAMETER NAME',
       'PARAMETER DETAIL', 'RUN NUMBER', 'BATCH NUMBER', 'DATE OF MANUFACTURE',
       'PROCESS START', 'PARAMETER DATE', 'LOCAL ID', 'VALUE_TEXT(Reported)',
       'VALUE_NUM(Recorded Full precision)', 'UOM', 'LAL', 'UAL', 'LCL', 'UCL',
       'LRL', 'URL', 'CL', 'NEL 1', 'NEL 2', 'NEL 3', 'NEL 4', 'PPK', 'REP',
       'CONTROL_PRECISION', 'ACTION_PRECISION', 'REJECTION_PRECISION', 'SCALE',
       'SUMMARY TABLE', 'DISPOSITION_DATE', 'RESULT_TYPE', 'SOURCE_SYSTEM',
       'CTRL_VIOLATION', 'ACTION_VIOLATION', 'REJ_VIOLATION'],
      dtype='object')

In [64]:
epogen.columns = map(str.lower, epogen.columns)

In [71]:
aran=pd.read_csv("Aranesp_quan.csv")

In [143]:
pd.unique(aran['spec status'])

array([ 0.,  1., -1., nan])

In [26]:
epogen=epogen.sort_values(by=['PARAMETER NAME','UNIT PROCEDURE','PARAMETER DATE'])

In [ ]:
ppk_single_row(values,ual,lal,spec_status)

In [18]:
pd.to_numeric("adsf","coerce")

nan

In [27]:
for para in quantitative_paras:
    for unit in unit_uniq:
        sub_df_len=len(epogen.loc[(epogen['PARAMETER NAME']==para) &(epogen['UNIT PROCEDURE']==unit)])
        if (sub_df_len>1):
           
            #could loop using the uniq combination of unit and para, otherwise add this row number check to avoid void error
            ual=epogen[(epogen['PARAMETER NAME']==para) &(epogen['UNIT PROCEDURE']==unit)].UAL.iloc[-1]
        
            lal=epogen[(epogen['PARAMETER NAME']==para) &(epogen['UNIT PROCEDURE']==unit)].LAL.iloc[-1]
            epogen["UAL"].loc[(epogen['PARAMETER NAME']==para) &(epogen['UNIT PROCEDURE']==unit)]=ual
            epogen["LAL"].loc[(epogen['PARAMETER NAME']==para) &(epogen['UNIT PROCEDURE']==unit)]=lal

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [68]:
epogen_quan=epogen[epogen["PARAMETER NAME"].isin(quantitative_paras)]

KeyError: 'PARAMETER NAME'

In [78]:
epogen_quan

,UNIT PROCEDURE,SAMPLING POINT,PARAMETER NAME,PARAMETER DETAIL,RUN NUMBER,BATCH NUMBER,DATE OF MANUFACTURE,PROCESS START,PARAMETER DATE,LOCAL ID,...,SCALE,SUMMARY TABLE,DISPOSITION_DATE,RESULT_TYPE,SOURCE_SYSTEM,CTRL_VIOLATION,ACTION_VIOLATION,REJ_VIOLATION,Spec Status,Ctrl Status
7473,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010195842,0010195842,2/1/2014 0:00,2/1/2014 0:00,2/1/2014 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7475,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010196613,0010196613,2/15/2014 0:00,2/15/2014 0:00,2/15/2014 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7477,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010235569,0010235569,2/15/2015 0:00,2/15/2015 0:00,2/15/2015 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7476,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010196676,0010196676,2/22/2014 0:00,2/22/2014 0:00,2/22/2014 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7478,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010235718,0010235718,2/22/2015 0:00,2/22/2015 0:00,2/22/2015 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7467,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010157616,0010157616,2/27/2013 0:00,2/27/2013 0:00,2/27/2013 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7474,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010196587,0010196587,2/8/2014 0:00,2/8/2014 0:00,2/8/2014 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7469,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010160720,0010160720,3/13/2013 0:00,3/13/2013 0:00,3/13/2013 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7479,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010237023,0010237023,3/2/2015 0:00,3/2/2015 0:00,3/2/2015 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1
7470,PRJU:APR6-PRDS-000687-AML,.,Anti-Contaminant Immunoassay,PRDS-000687,0010161089,0010161089,3/24/2013 0:00,3/24/2013 0:00,3/24/2013 0:00,.,...,LIN,.,.,Number,SMLIMS,.,.,.,1,1


In [22]:
l = ['sat', 'bat', 'cat', 'mat'] 
# map() can listify the list of strings individually 
map(list, l)

In [185]:
def skewness(values):
    n=len(values)
    if (n<3):
        result=np.nan
    else:
        sample_mean=np.mean(values)
        sample_std=np.std(values,ddof=1)
        result=n*np.sum(((values-sample_mean)/sample_std)**3)/((n-1)*(n-2))
    return(result)

In [ ]:
    def ppk_single_row(self, values,ual,lal,spec_status):
        if spec_status==0:
            ppk=np.minimum(float(ual)-np.mean(values),np.mean(values)-float(lal) )/(3*np.std(values,ddof=1))
        else:
            if spec_status==1:
                ppk=(float(ual)-np.mean(values))/(3*np.std(values,ddof=1))
            else:
                if spec_status==-1:
                    ppk=(np.mean(values)-float(lal))/(3*np.std(values,ddof=1))
                else:
                    ppk="NA3"
        return(ppk)


In [200]:
def shapiro_func(values):
    if (len(values)<4):
        return(np.nan)
    else:
        return(shapiro(values)[1])
              
         
def kstest_func( values):
    if len(values<4):
        return(np.nan)
    else:
        return(kstest(values, "norm")[1])

In [209]:
def kurt(values):
    n=len(values)
    if (n<4): 
        result=np.nan
    else:
        sample_mean=np.mean(values)
        sample_std=np.std(values,ddof=1)
        result=np.sum(((values-sample_mean)/sample_std)**4)*n*(n+1)/((n-1)*(n-2)*(n-3))-3*(n-1)**2/((n-2)*(n-3))
    return(result)

In [211]:
def CV(values):
    sample_mean=np.mean(values)
    sample_std=np.std(values,ddof=1)
    return(100*sample_std/sample_mean)

In [212]:
aggre_summary_df_generator(aran,input_type="")

CV  \
parameter name                                     unit procedure                 
CHO Proteins Immunoassay                           PRJU:APR6-DS-AML   41.010848   
Carbohydrate Profile (sialo) oligosaccharide ma... PRJU:APR6-DS-AML    0.406578   
Carbohydrate Profile (sialo) oligosaccharide ma... PRJU:APR6-DS-AML    1.779005   
Carbohydrate Profile (sialo) oligosaccharide ma... PRJU:APR6-DS-AML    5.777439   
Chloride                                           IEDL:ADL9-DP-ADL    1.223824   
                                                   PRJU:APR1-DP-ADL    1.145080   
                                                   PRJU:APR1-DP-AML    1.136793   
                                                   PRJU:APR1-DP-CRL    1.742658   
                                                   PRJU:APR14-DP-ADL   1.371914   
                                                   PRJU:APR14-DP-AML   2.300021   
                                                   USTO:20-DP-ATO      2.647219   
Immunoassay                                        IEDL:ADL9-DP-ADL    5.169090   
                                                   PRJU:APR1-DP-ADL    7.639733   
                                                   PRJU:APR1-DP-AML    6.427982   
                                                   PRJU:APR1-DP-CRL    3.526504   
                                                   PRJU:APR14-DP-ADL   5.828434   
                                                   PRJU:APR14-DP-AML   7.011518   
                                                   USTO:20-DP-ATO      7.216002   
In Vitro Bioassay                                  PRJU:APR6-DS-AML    6.941254   
In vitro bioassay                                  IEDL:ADL9-DP-ADL    5.821578   
                                                   PRJU:APR1-DP-ADL    5.974024   
                                                   PRJU:APR1-DP-AML    8.044417   
                                                   PRJU:APR1-DP-CRL    5.047273   
                                                   PRJU:APR14-DP-ADL   5.581233   
                                                   PRJU:APR14-DP-AML   6.769522   
                                                   USTO:20-DP-ATO      7.497938   
Isoelectric Focusing: Isoform 21 + 22              PRJU:APR6-DS-AML    6.331834   
Isoelectric Focusing: Isoform 22                   PRJU:APR6-DS-AML   12.806489   
Isoelectric Focusing: Isoforms 19 + 20             PRJU:APR6-DS-AML   18.458627   
Isoelectric Focusing: Isoforms 19 to 22            PRJU:APR6-DS-AML    0.435272   
...                                                                         ...   
Volume (0.5 mL Syringe)                            PRJU:APR1-DP-CRL    0.827371   
                                                   PRJU:APR14-DP-ADL   0.385012   
                                                   PRJU:APR14-DP-AML   0.535836   
Volume (0.6 mL Syringe)                            PRJU:APR1-DP-ADL    0.594147   
                                                   PRJU:APR1-DP-AML    0.631257   
                                                   PRJU:APR1-DP-CRL    0.612585   
                                                   PRJU:APR14-DP-ADL   0.343870   
                                                   PRJU:APR14-DP-AML   0.454668   
Volume (0.65 mL Syringe)                           PRJU:APR1-DP-ADL    0.730275   
                                                   PRJU:APR1-DP-AML    0.730792   
                                                   PRJU:APR1-DP-CRL    1.040407   
                                                   PRJU:APR14-DP-ADL   0.311239   
                                                   PRJU:APR14-DP-AML   0.574830   
Volume (0.75 mL Vial)                              PRJU:APR1-DP-AML    1.601399   
Volume (1.0 mL Syringe)                            PRJU:APR1-DP-ADL    0.498219   
                                                   PRJU:APR1-DP-AML    0.492228   
          

In [205]:
def aggre_summary_df_generator(df, input_type, min_size=30):
        
        #input_type:"" OR "_no_OOS"
        
        #initialize the summary output df
        #aggre_stats_output_df=pd.DataFrame(columns=["parameter name",'unit procedure',#"PRDS",#\
                                              # "N","mean","median","STD","variance","UAL","LAL","spec status","Ppk"+input_type,\
                                             #  "skewness","kurtosis","CV",#"anderson-darling"\
                                               # "shapiro","KS"\
                                                #  ])
        
    aggre_stats_output_df=pd.DataFrame(data={\
                "N":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].count(),\
              "mean":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].mean(),\
          "median":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].median(),\
          "STD":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].std(),\
          "varaince":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].var(),\
          "Ppk"+input_type:df.groupby(["parameter name","unit procedure"]).apply(Ppk_groupby,min_size=30),\
          "skewness":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].apply(skewness),\
          "kurtosis":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].apply(kurt),\
          "CV":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].apply(CV),\
          "shapiro":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].apply(shapiro_func),\
          "KS":df.groupby(["parameter name","unit procedure"])["value_num(recorded/full precision)"].apply(kstest_func),\
          "spec status":df.groupby(["parameter name","unit procedure"])["spec status"].last(),\
          "UAL":df.groupby(["parameter name","unit procedure"])["UAL"].last(),\
          "LAL":df.groupby(["parameter name","unit procedure"])["LAL"].last()\
                                            })
    return(aggre_stats_output_df)
        
        
        
        
        


In [204]:
def Ppk_groupby(df, min_size):
    
    spec_status=df["spec status"].iloc[0]
    ual=df.UAL.iloc[0]
    lal=df.LAL.iloc[0]
    values=df['value_num(recorded/full precision)']
    
    if (len(values)<min_size):
                ppk="NA1"
            
    if( (len(values)>=min_size) and (len(pd.unique(values)<5)) ):
                ppk="NA2"
            
    if( (len(values)>=min_size) and (len(pd.unique(values)>=5)) ):
        
        if spec_status==0:
            ppk=np.minimum(float(ual)-np.mean(values),np.mean(values)-float(lal) )/(3*np.std(values,ddof=1))
        else:
            if spec_status==1:
                ppk=(float(ual)-np.mean(values))/(3*np.std(values,ddof=1))
            else:
                if spec_status==-1:
                    ppk=(np.mean(values)-float(lal))/(3*np.std(values,ddof=1))
                else:
                    ppk="NA3"
    #return a single value will make it similar to std, mean function, easy for group by but hard for generating summary tables
    #can also do: df[ppk]=ppk, return(df), the df will be changed outside the function. same ppk populated for each group
    #Raw ppk(with oos)
    df["Ppk"]=ppk
    return(ppk)

In [109]:
test.to_csv("groupby.csv")

In [67]:
epogen_quan

NameError: name 'epogen_quan' is not defined

In [107]:
epogen_pmaf_summary=pd.DataFrame(columns=['parameter name', 'unit procedure', 'UAL', 'LAL', 'spec status', 'N_x',
       'Ppk', 'Ppk_noOOST', 'N_y', 'batch number(exclusion type)'])

In [89]:
epogen_only_pmaf=epogen[(epogen["UNIT PROCEDURE"]=="PRJU:APR1-PRDS-000874-AML")&\
                        (epogen["PARAMETER NAME"]=="EPO Gene Expression Bioassay")]


In [110]:
epogen_only_pmaf

,UNIT PROCEDURE,SAMPLING POINT,PARAMETER NAME,PARAMETER DETAIL,RUN NUMBER,BATCH NUMBER,DATE OF MANUFACTURE,PROCESS START,PARAMETER DATE,LOCAL ID,...,ACTION_PRECISION,REJECTION_PRECISION,SCALE,SUMMARY TABLE,DISPOSITION_DATE,RESULT_TYPE,SOURCE_SYSTEM,CTRL_VIOLATION,ACTION_VIOLATION,REJ_VIOLATION
4821,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010233466,0010233466,1/22/2015 0:00,1/22/2015 0:00,1/22/2015 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4849,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010309454,0010309454,10/4/2016 0:00,10/4/2016 0:00,10/4/2016 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4831,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010266626,0010266626,11/11/2015 0:00,11/11/2015 0:00,11/11/2015 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4820,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010228292,0010228292,12/8/2014 0:00,12/8/2014 0:00,12/8/2014 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4832,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010279477,0010279477,2/10/2016 0:00,2/10/2016 0:00,2/10/2016 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4822,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010235537,0010235537,2/5/2015 0:00,2/5/2015 0:00,2/5/2015 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4833,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010279947,0010279947,3/29/2016 0:00,3/29/2016 0:00,3/29/2016 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4824,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010240920,0010240920,4/14/2015 0:00,4/14/2015 0:00,4/14/2015 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4825,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010245695,0010245695,4/30/2015 0:00,4/30/2015 0:00,4/30/2015 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.
4823,PRJU:APR1-PRDS-000874-AML,.,EPO Gene Expression Bioassay,PRDS-000874,0010245291,0010245291,4/9/2015 0:00,4/9/2015 0:00,4/9/2015 0:00,.,...,3,.,LIN,.,.,Number,SMLIMS,.,.,.


In [108]:
epogen_pmaf_summary=epogen_pmaf_summary.append({
        'parameter name':epogen_only_pmaf["PARAMETER NAME"].iloc[0], \
        'unit procedure':epogen_only_pmaf["UNIT PROCEDURE"].iloc[0], \
        'UAL':float(epogen_only_pmaf.UAL.iloc[-1]), 'LAL':float(epogen_only_pmaf.UAL.iloc[-1]), \
        'spec status':0, 'N_x':30,\
       'Ppk':0.69687, 'Ppk_noOOST':0.83606, 'N_y':29, 'batch number(exclusion type)':"0010298785(OOS)"

            },\
    ignore_index=True
        )
epogen_pmaf_summary.to_csv("epogen_pmaf_summary.csv")

In [91]:
ppk_single_row(epogen_only_pmaf["VALUE_NUM(Recorded Full precision)"],24000,16000,0)

0.6968716725233876

In [98]:
ppk_single_row(epogen_only_pmaf_after["VALUE_NUM(Recorded Full precision)"],24000,16000,0)

0.8360574014125609

In [97]:
epogen_only_pmaf_after=epogen_only_pmaf[(epogen_only_pmaf["VALUE_NUM(Recorded Full precision)"]>\
                                         float(epogen_only_pmaf.LAL.iloc[-1]))\
                 &(epogen_only_pmaf["VALUE_NUM(Recorded Full precision)"]<float(epogen_only_pmaf.UAL.iloc[-1]))]

In [116]:
epogen_only_pmaf_after.to_csv("epogen-EPO-JMP.csv",index=False)

In [102]:
epogen_only_pmaf.loc[epogen_only_pmaf.index.difference(epogen_only_pmaf_after.index)]["BATCH NUMBER"]

4842    0010298785
Name: BATCH NUMBER, dtype: object

In [65]:
epogen.UAL.str.extract('(\d+)')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


7473    0
7475    0
7477    0
7476    0
7478    0
7467    0
7474    0
7469    0
7479    0
7470    0
7471    0
7468    0
7480    0
7472    0
7874    0
7875    0
7860    0
7873    0
7872    0
7883    0
7884    0
7861    0
7876    0
7885    0
7862    0
7877    0
7886    0
7863    0
7878    0
7865    0
       ..
8108    7
8109    7
8094    7
8107    7
8106    7
8117    7
8118    7
8095    7
8110    7
8119    7
8096    7
8111    7
8120    7
8097    7
8112    7
8099    7
8098    7
8100    7
8121    7
8101    7
8114    7
8103    7
8102    7
8113    7
8122    7
8115    7
8116    7
8105    7
8104    7
8123    7
Name: UAL, Length: 8274, dtype: object